In [29]:
import requests, pandas as pd

In [30]:
#    "in": "state:06",   # California
URL = "https://api.census.gov/data/2022/abscs"
params = {
    "get": "NAME,GEO_ID,NAICS2022,NAICS2022_LABEL,FIRMPDEMP,EMP,PAYANN,RCPPDEMP",
    "for": "county:*",
    "INDLEVEL": "2"     # 2-digit NAICS
}

r = requests.get(URL, params=params, timeout=60)
r.raise_for_status()
hdr, *rows = r.json()
df = pd.DataFrame(rows, columns=hdr)

In [34]:
# Cast numerics
for c in ["FIRMPDEMP","EMP","PAYANN","RCPPDEMP"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
    df["year_num"] = pd.to_numeric("2022", errors="coerce")

# Nice ordering
#df = df.sort_values(["state","county","NAICS2022"]).reset_index(drop=True)

In [35]:
df = df.rename(columns={
    'NAME': 'cnty_nm',
    'GEO_ID': 'geo_id',
    'NAICS2022': 'naics2_sector_cd',
    'NAICS2022_LABEL': 'naics2_sector_desc',
    'INDLEVEL': 'ind_level_num',
    'FIRMPDEMP': 'abs_firm_num',          # firms per employer → firm count
    'EMP': 'abs_emp_num',                 # employment count
    'PAYANN': 'abs_payroll_usd_amt',      # annual payroll (USD amount)
    'RCPPDEMP': 'abs_rcpt_per_emp_usd_amt',   # receipts per employee (USD)
    'state': 'state_fips_cd',
    'county': 'cnty_fips_cd'
})

# Create combined fips cd
df["state_cnty_fips_cd"] = df["state_fips_cd"] + df["cnty_fips_cd"]


In [36]:
df

,cnty_nm,geo_id,naics2_sector_cd,naics2_sector_desc,abs_firm_num,abs_emp_num,abs_payroll_usd_amt,abs_rcpt_per_emp_usd_amt,ind_level_num,state_fips_cd,cnty_fips_cd,year_nm,year_num,state_cnty_fips_cd
0,"Autauga County, Alabama",0500000US01001,00,Total for all sectors,0,0,0,0,2,01,001,2022,2022,01001
1,"Autauga County, Alabama",0500000US01001,11,"Agriculture, forestry, fishing and hunting",0,0,0,0,2,01,001,2022,2022,01001
2,"Autauga County, Alabama",0500000US01001,21,"Mining, quarrying, and oil and gas extraction",0,0,0,0,2,01,001,2022,2022,01001
3,"Autauga County, Alabama",0500000US01001,22,Utilities,5,133,17854,191100,2,01,001,2022,2022,01001
4,"Autauga County, Alabama",0500000US01001,23,Construction,118,387,18215,83338,2,01,001,2022,2022,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56559,"Weston County, Wyoming",0500000US56045,56,Administrative and support and waste managemen...,0,0,0,0,2,56,045,2022,2022,56045
56560,"Weston County, Wyoming",0500000US56045,62,Health care and social assistance,0,0,0,0,2,56,045,2022,2022,56045
56561,"Weston County, Wyoming",0500000US56045,71,"Arts, entertainment, and recreation",2,4,113,422,2,56,045,2022,2022,56045
56562,"Weston County, Wyoming",0500000US56045,72,Accommodation and food services,0,0,0,0,2,56,045,2022,2022,56045


In [37]:
df.columns.tolist()


['cnty_nm',
 'geo_id',
 'naics2_sector_cd',
 'naics2_sector_desc',
 'abs_firm_num',
 'abs_emp_num',
 'abs_payroll_usd_amt',
 'abs_rcpt_per_emp_usd_amt',
 'ind_level_num',
 'state_fips_cd',
 'cnty_fips_cd',
 'year_nm',
 'year_num',
 'state_cnty_fips_cd']

In [43]:
column_reorder = ['year_num', 'state_cnty_fips_cd', 'naics2_sector_cd', 'cnty_nm', 'geo_id', 'naics2_sector_desc', 'ind_level_num', 'state_fips_cd', 'cnty_fips_cd', 'abs_firm_num', 'abs_emp_num', 'abs_payroll_usd_amt', 'abs_rcpt_per_emp_usd_amt']

In [44]:
df = df[column_reorder]

In [45]:
print(df.head())  # preview

   year_num state_cnty_fips_cd naics2_sector_cd                  cnty_nm  \
0      2022              01001               00  Autauga County, Alabama   
1      2022              01001               11  Autauga County, Alabama   
2      2022              01001               21  Autauga County, Alabama   
3      2022              01001               22  Autauga County, Alabama   
4      2022              01001               23  Autauga County, Alabama   

           geo_id                             naics2_sector_desc  \
0  0500000US01001                          Total for all sectors   
1  0500000US01001     Agriculture, forestry, fishing and hunting   
2  0500000US01001  Mining, quarrying, and oil and gas extraction   
3  0500000US01001                                      Utilities   
4  0500000US01001                                   Construction   

  ind_level_num state_fips_cd cnty_fips_cd  abs_firm_num  abs_emp_num  \
0             2            01          001             0     

In [46]:
df.to_csv("/Users/michaelwalker/RDM_Datalab/rdm-datalab-pipelines/data_clean/abs/econ_bnchmrk_2022_abs.csv", index=False)
print("Saved: econ_bnchmrk_2022_abs.csv")

Saved: econ_bnchmrk_2022_abs.csv


In [10]:
df.loc[0]

NAME               Autauga County, Alabama
GEO_ID                      0500000US01001
NAICS2022                               00
NAICS2022_LABEL      Total for all sectors
INDLEVEL                                 2
FIRMPDEMP                                0
EMP                                      0
PAYANN                                   0
RCPPDEMP                                 0
INDLEVEL                                 2
state                                   01
county                                 001
Name: 0, dtype: object

In [8]:
df.head(1)

,NAME,GEO_ID,NAICS2022,NAICS2022_LABEL,INDLEVEL,FIRMPDEMP,EMP,PAYANN,RCPPDEMP,INDLEVEL,state,county
0,"Autauga County, Alabama",0500000US01001,00,Total for all sectors,2,0,0,0,0,2,01,001


In [11]:
df[["GEO_ID","NAICS2022"]].value_counts()

GEO_ID          NAICS2022
0500000US01001  00           1
0500000US39101  62           1
                31-33        1
                42           1
                44-45        1
                            ..
0500000US21107  21           1
                22           1
                23           1
                31-33        1
0500000US56045  81           1
Name: count, Length: 56564, dtype: int64

In [12]:
filtered_df = df[df['NAME'] == 'Davidson County, Tennessee']

In [13]:
filtered_df.head(10)

,NAME,GEO_ID,NAICS2022,NAICS2022_LABEL,INDLEVEL,FIRMPDEMP,EMP,PAYANN,RCPPDEMP,INDLEVEL,state,county
44174,"Davidson County, Tennessee",0500000US47037,00,Total for all sectors,2,16486,546020,36582320,218032051,2,47,037
44175,"Davidson County, Tennessee",0500000US47037,11,"Agriculture, forestry, fishing and hunting",2,5,23,861,1816,2,47,037
44176,"Davidson County, Tennessee",0500000US47037,21,"Mining, quarrying, and oil and gas extraction",2,5,130,10288,106393,2,47,037
44177,"Davidson County, Tennessee",0500000US47037,22,Utilities,2,12,175,13358,222401,2,47,037
44178,"Davidson County, Tennessee",0500000US47037,23,Construction,2,1134,22160,1585085,8795484,2,47,037
44179,"Davidson County, Tennessee",0500000US47037,31-33,Manufacturing,2,523,21613,1277392,8083899,2,47,037
44180,"Davidson County, Tennessee",0500000US47037,42,Wholesale trade,2,1108,28063,2256219,52048802,2,47,037
44181,"Davidson County, Tennessee",0500000US47037,44-45,Retail trade,2,1910,45769,1796921,25928823,2,47,037
44182,"Davidson County, Tennessee",0500000US47037,48-49,Transportation and warehousing,2,546,26261,1392348,6421215,2,47,037
44183,"Davidson County, Tennessee",0500000US47037,51,Information,2,530,17320,1485584,7984391,2,47,037


In [16]:
df.columns

Index(['NAME', 'GEO_ID', 'NAICS2022', 'NAICS2022_LABEL', 'INDLEVEL',
       'FIRMPDEMP', 'EMP', 'PAYANN', 'RCPPDEMP', 'INDLEVEL', 'state',
       'county'],
      dtype='object')

In [18]:
df

,cnty_nm,geo_id,naics2022_sector_cd,naics2022_sector_desc,ind_level_num,abs_firm_num,abs_emp_num,abs_payroll_usd_amt,abs_rcpt_per_emp_usd_amt,ind_level_num,state_fips_cd,cnty_fips_cd
0,"Autauga County, Alabama",0500000US01001,00,Total for all sectors,2,0,0,0,0,2,01,001
1,"Autauga County, Alabama",0500000US01001,11,"Agriculture, forestry, fishing and hunting",2,0,0,0,0,2,01,001
2,"Autauga County, Alabama",0500000US01001,21,"Mining, quarrying, and oil and gas extraction",2,0,0,0,0,2,01,001
3,"Autauga County, Alabama",0500000US01001,22,Utilities,2,5,133,17854,191100,2,01,001
4,"Autauga County, Alabama",0500000US01001,23,Construction,2,118,387,18215,83338,2,01,001
...,...,...,...,...,...,...,...,...,...,...,...,...
56559,"Weston County, Wyoming",0500000US56045,56,Administrative and support and waste managemen...,2,0,0,0,0,2,56,045
56560,"Weston County, Wyoming",0500000US56045,62,Health care and social assistance,2,0,0,0,0,2,56,045
56561,"Weston County, Wyoming",0500000US56045,71,"Arts, entertainment, and recreation",2,2,4,113,422,2,56,045
56562,"Weston County, Wyoming",0500000US56045,72,Accommodation and food services,2,0,0,0,0,2,56,045
